<img src='./img/EU-Copernicus-EUM_3Logos.png' alt='Logo EU Copernicus EUMETSAT' align='right' width='50%'></img>

<br>

# LTPy functions

In [23]:
import os
from matplotlib import pyplot as plt
import h5py

import xarray as xr
import numpy as np

from matplotlib import pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

### <a id='visualize_gome_l2'></a>`visualize_gome_l2`

In [31]:
def visualize_gome_l2(xr_dataarray, conversion_factor, projection, vmin, vmax, color_scale,set_global=False):
    """ 
    Visualizes a xarray DataArray in a given projection using matplotlib's scatter function.
    
    Parameters:
        xr_dataarray(xarrayDataArray): a one-dimensional xarray DataArray object with latitude and longitude information as coordinates
        conversion_factor(float): any number to convert the DataArray values
        projection: choose one of cartopy's projection, e.g. ccrs.PlateCarree()
        vmin(int): minimum number on visualisation legend
        vmax(int): maximum number on visualisation legend
        color_scale(str): string taken from matplotlib's color ramp reference
        set_global: set True, if the plot shall have a global coverage
    """
    fig, ax = plt.subplots(figsize=(40, 10))
    ax = plt.axes(projection=projection)

    ax.coastlines()
    if set_global:
        ax.set_global()

    gl = ax.gridlines(draw_labels=True, linestyle='--')
    gl.xlabels_top=False
    gl.ylabels_right=False
    gl.xformatter=LONGITUDE_FORMATTER
    gl.yformatter=LATITUDE_FORMATTER
    
    gl.xlabel_style={'size':14}
    gl.ylabel_style={'size':14}

    # plot pixel positions
    img = ax.scatter(
        xr_dataarray.longitude.data,
        xr_dataarray.latitude.data,
        c=xr_dataarray.data*conversion_factor,
        cmap=plt.cm.get_cmap(color_scale),
        marker='o',
        s=0.5,
        transform=ccrs.PlateCarree(),
        vmin=vmin,
        vmax=vmax
    )

    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.xlabel("Longitude", fontsize=16)
    plt.ylabel("Latitude", fontsize=16)
    cbar = fig.colorbar(img, ax=ax, orientation='horizontal', fraction=0.04, pad=0.1)
    cbar.set_label(xr_dataarray.units, fontsize=16)
    cbar.ax.tick_params(labelsize=14)
    ax.set_title(xr_dataarray.long_name, fontsize=20, pad=20.0)
    plt.show()

### <a id='load_l2_data_xr'></a>`load_l2_data_xr`

In [27]:
import os
def load_l2_data_xr(directory, internal_filepath, parameter, longname, unit, name):
    """ 
    Loads a Metop-A/B Level 2 dataset in HDF format and returns a xarray DataArray with all the ground pixels of all GOME-2A files for one specific day
    
    Parameters:
        directory: directory where the HDF files are stored
        internal_filepath: internal path of the data file that is of interest, e.g. TOTAL_COLUMNS
        parameter: paramter that is of interest, e.g. NO2
        longname: Descriptive name of the variable stored in the resulting xarray DataArray
        unit: Unit of the parameter
        name: short name of the variable
    
    Returns:
        1-dimensional xarray DataArray with latitude / longitude information as coordinate information
    """
    fileList = os.listdir(directory)
    datasets = []

    for i in fileList:
        fullpath=os.path.join(directory, i)
        tmp = h5py.File(fullpath, 'r')
        groups =tmp.keys()
        tmp[internal_filepath]
        latitude = tmp['GEOLOCATION/LatitudeCentre']
        longitude = tmp['GEOLOCATION/LongitudeCentre']
        param = tmp[internal_filepath+'/'+parameter]
        da_tmp = xr.DataArray(
            param[()],
            dims=('ground_pixel'),
            coords={
                'latitude': ('ground_pixel', latitude[:]),
                'longitude': ('ground_pixel', longitude[:])
            },
            attrs={'long_name': longname, 'units': unit},
            name=name
        )
        datasets.append(da_tmp)

    return xr.concat(datasets, dim='ground_pixel')

### <a id='visualize_gome_mollweide'></a>`visualize_gome_mollweide`

In [26]:
def visualize_gome_mollweide(xr_dataarray, conversion_factor, color_scale, vmin, vmax):
    """ 
    Visualizes a xarray dataarray in a mollweide projection using matplotlib's scatter function.
    
    Parameters:
        xr_dataarray(xarrayDataArray): a three-dimensional xarray DataArray object
        conversion_factor(float): any number to convert the DataArray values
        color_scale(str): string taken from matplotlib's color ramp reference  
        vmin(int): minimum number on visualisation legend
        vmax(int): maximum number on visualisation legend
    """
    fig, ax = plt.subplots(figsize=(40, 10))
    ax = plt.axes(projection=ccrs.Mollweide())

    ax.coastlines()
    ax.set_global()

    ax.gridlines(linestyle='--')
    img = ax.scatter(
        xr_dataarray.longitude.data,
        xr_dataarray.latitude.data,
        c=xr_dataarray.data*conversion_factor,
        cmap=plt.cm.get_cmap(color_scale),
        marker='o',
        s=0.5,
        transform=ccrs.PlateCarree(),
        vmin=vmin,
        vmax=vmax
    )

    cbar = fig.colorbar(img, ax=ax, orientation='horizontal', fraction=0.04, pad=0.1)
    cbar.set_label(xr_dataarray.units, fontsize=16)
    cbar.ax.tick_params(labelsize=14)
    ax.set_title(xr_dataarray.long_name, fontsize=20, pad=20.0)
    plt.show()

### <a id='visualize_imshow'></a>`visualize_imshow`

In [13]:
def visualize_imshow(data_array, projection, extent, conversion_factor, color_scale, vmin, vmax, set_global=False):
    """ 
    Visualizes a numpy MaskedArray with matplotlib's 'imshow' function.
    
    Parameters:
        data_array: any numpy MaskedArray, e.g. loaded with the NetCDF library and the Dataset function
        projection: a projection provided by the cartopy library, e.g. ccrs.PlateCarree()
        extent: list with the following order - [min(lon),max(lon),max(lat),min(lat)]
        conversion_factor(float): any number to convert the DataArray values
        color_scale(str): string taken from matplotlib's color ramp reference  
        vmin(int): minimum number on visualisation legend
        vmax(int): maximum number on visualisation legend
        set_global: set True, if the plot shall have a global coverage
    """
    fig=plt.figure(figsize=(20, 12))

    ax=plt.axes(projection=projection)
    ax.coastlines()
    ax.gridlines()
    if(set_global):
        ax.set_global()

    ax.set_title(data_array.long_name, fontsize=20, pad=20.0)

    gl = ax.gridlines(draw_labels=True, linestyle='--')
    gl.xlabels_top=False
    gl.ylabels_right=False
    gl.xformatter=LONGITUDE_FORMATTER
    gl.yformatter=LATITUDE_FORMATTER
    
    gl.xlabel_style={'size':14}
    gl.ylabel_style={'size':14}

    img1 = plt.imshow(data_array[:]*conversion_factor,
           extent=extent,
           cmap=color_scale,
           aspect='auto',
           vmin=vmin,
           vmax=vmax)

    cbar = fig.colorbar(img1, ax=ax, orientation='horizontal', fraction=0.04, pad=0.1)
    cbar.set_label(str(conversion_factor) + ' ' + data_array.units, fontsize=16)
    cbar.ax.tick_params(labelsize=14)
    
    plt.show()

### <a id='visualize_s5p_pcolormesh'></a>`visualize_s5p_pcolormesh`

In [ ]:
def visualize_s5p_pcolormesh(data_array, longitude, latitude, projection, color_scale, unit, long_name, lonmin, lonmax, latmin, latmax, set_global=True):
    """ 
    Visualizes a numpy array with matplotlib's 'pcolormesh' function.
    
    Parameters:
        data_array: any numpy MaskedArray, e.g. loaded with the NetCDF library and the Dataset function
        longitude:
        latitude:
        projection: a projection provided by the cartopy library, e.g. ccrs.PlateCarree()
        color_scale(str): string taken from matplotlib's color ramp reference
        unit(str): the unit of the pararmeter
        long_name(str): long name of the parameter, taken from the NetCDF file if possible
        longmin,lonmax,latmin,latmax: geographic extent of the plot
        set_global: set True, if the plot shall have a global coverage
    """
    fig=plt.figure(figsize=(20, 10))

    ax = plt.axes(projection=projection)

    # define the coordinate system that the grid lons and grid lats are on
    img = plt.pcolormesh(longitude, latitude, np.squeeze(data_array), norm=LogNorm(), cmap=plt.get_cmap(color_scale), transform=ccrs.PlateCarree())

    ax.coastlines()

    if(set_global):
        ax.set_global()
        ax.gridlines()
    else:
        ax.set_extent([lonmin, lonmax, latmin, latmax], projection)
        gl = ax.gridlines(draw_labels=True, linestyle='--')
        gl.xlabels_top=False
        gl.ylabels_right=False
        gl.xformatter=LONGITUDE_FORMATTER
        gl.yformatter=LATITUDE_FORMATTER
        gl.xlabel_style={'size':12}
        gl.ylabel_style={'size':12}

    cbar = fig.colorbar(img, ax=ax, orientation='horizontal', fraction=0.04, pad=0.1)
    cbar.set_label(unit, fontsize=16)
    cbar.ax.tick_params(labelsize=14)
    ax.set_title(long_name, fontsize=20, pad=20.0)

    plt.show()

### <a id='generate_geographical_subset'></a>`generate_geographical_subset`

In [1]:
def generate_geographical_subset(xarray, latmin, latmax, lonmin, lonmax):
    """ 
    Generates a geographical subset of a xarray DataArray and shifts the longitude grid from a 0-360 to a -180 to 180 deg grid.
    
    Parameters:
        xarray: a xarray DataArray with latitude and longitude coordinates
        latmin, latmax, lonmin, lonmax (int): boundaries of the geographical subset
        
    Returns:
        Geographical subset of a xarray DataArray.
    """   
    xarray = xarray.assign_coords(longitude=(((xarray.longitude + 180) % 360) - 180))
    return xarray.where((xarray.latitude < latmax) & (xarray.latitude > latmin) & (xarray.longitude < lonmax) & (xarray.longitude > lonmin),drop=True)

### <a id='generate_masked_array'></a>`generate_masked_array`

In [12]:
def generate_masked_array(xarray, mask, threshold):
    """ 
    Applies a cloud mask (e.g. cloud fraction values) onto a given data array, based on a given threshold.
    
    Parameters:
        xarray: a three-dimensional xarray DataArray object
        mask: 1-dimensional xarray DataArray, e.g. cloud fraction values
        threshold (float): any number between 0 and 1, specifying the degree of cloudiness which is acceptable.
        
    Returns:
        Masked xarray DataArray with flagged negative values
    """
    cloud_mask = xr.where(mask < threshold, 1, 0) #Generate cloud mask with value 1 for the pixels we want to keep
    xarray_masked = xr.where(cloud_mask ==1, xarray, 0) #Apply mask onto the DataArray
    xarray_masked.attrs = xarray.attrs #Set DataArray attributes 
    return xarray_masked[xarray_masked > 0] #Return masked DataArray and flag negative values

### <a id='load_masked_l2_da'></a>`load_masked_l2_da`

In [2]:
def load_masked_l2_da(directory, internal_filepath, parameter, longname, unit, name, threshold):
    """ 
    Loads a Metop-A/B Gome-2 Level 2 data and cloud fraction information and returns a masked data array.
    
    Parameters:
        directory(str): Path to directory with Level 2 data files.
        internal_filepath(str): Internal file path under which the parameters are strored, e.g. TOTAL_COLUMNS
        parameter(str): atmospheric parameter, e.g. NO2
        longname(str): long name of the parameter that shall be used
        unit(str): unit of the parameter
        name(str): short name of the parameter
        
    Returns:
        Masked xarray DataArray with flagged negative values
    """  
    da = load_l2_data_xr(directory, internal_filepath, parameter, longname, unit, name)
    cloud_fraction = load_l2_data_xr(directory, 'CLOUD_PROPERTIES', 'CloudFraction', '-', 'Cloud Fraction', 'cloud_fraction')
    
    return generate_masked_array(da, cloud_fraction, threshold)